In [ ]:
import rasterio as rio
from rasterio.warp import reproject, Resampling, calculate_default_transform
import sys
import re

In [ ]:
def coregister(template, input, out_path):
    with rio.open(input) as input:
        with rio.open(template) as template:
            out_crs = template.crs
            out_transform, out_width, out_height = calculate_default_transform(
                input.crs,
                out_crs,
                template.width,
                template.height,
                *template.bounds,
            )
        out_kwargs = input.meta.copy()
        out_kwargs.update({
            "crs": out_crs,
            "transform": out_transform,
            "width": out_width,
            "height": out_height,
            "nodata": None
        })
        with rio.open(out_path, "w", **out_kwargs) as output:
            for i in range(1, input.count + 1): reproject(
                    source = rio.band(input, i),
                    destination = rio.band(output, i),
                    src_transform = input.transform,
                    src_crs = input.crs,
                    dst_transform = out_transform,
                    dst_crs = out_crs,
                    resampling = Resampling.nearest
                )